# 02 Homework: Vector Search

In [1]:
!python -m pip install -U pip
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [2]:
import numpy as np

from fastembed import TextEmbedding

## Q1. Embedding the query

In [3]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

embedding_model = TextEmbedding(model_name=model_handle)

In [4]:
query1 = 'I just discovered the course. Can I join now?'
documents1 = [ query1 ]
q1 = np.array(list(embedding_model.embed(documents1))[0])
print('Min value: ', sorted(q1)[0])

Min value:  -0.11726373885183883


In [5]:
np.linalg.norm(q1)

np.float64(1.0)

In [6]:
q1.dot(q1)

np.float64(1.0000000000000002)

## Q2. Cosine similarity with another vector

In [7]:
query2 = 'Can I still join the course after the start date?'
documents2 = [ query2 ]
q2 = np.array(list(embedding_model.embed(documents2))[0])
print('cosine similarity: ', q1.dot(q2))

cosine similarity:  0.9008528895674548


## Q3. Ranking by cosine

In [8]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [9]:
documents_text = [ doc['text'] for doc in documents ]
embeddings = np.array(list(embedding_model.embed(documents_text)))

In [10]:
similarities = embeddings.dot(q1)
print('Document index with the highest similarity: ', int(similarities.argsort()[-1]))

Document index with the highest similarity:  1


## Q4. Ranking by cosine, version two

In [11]:
documents_full_text = [ doc['question'] + ' ' + doc['text'] for doc in documents ]
embeddings_full_text = np.array(list(embedding_model.embed(documents_full_text)))

In [12]:
similarities_full_text = embeddings_full_text.dot(q1)
print('Document index with the highest similarity: ', int(similarities_full_text.argsort()[-1]))

Document index with the highest similarity:  0


## Q5. Selecting the embedding model

In [13]:
print('Smallest dimensionality: ', sorted(TextEmbedding.list_supported_models(), key=lambda item: item['dim'])[0]['dim'])

Smallest dimensionality:  384


## Q6. Indexing with qdrant (2 points)

In [14]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [15]:
from qdrant_client import QdrantClient, models

In [16]:
client = QdrantClient("http://localhost:6333") 

In [18]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"

collection_name = "llm-zoomcamp-rag"

client.delete_collection(collection_name=collection_name)
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [19]:
points = []

for id, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=id,
        vector=vector,
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        }
    )
    points.append(point)

In [20]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle 
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [22]:
print('Highest score: ', search(query1).points[0].score)

Highest score:  0.87031734
